# Computing with Riemann Surfaces

First, we construct the Riemann surface, $X$, defined by the complex algebraic curve

$$
f(x,y) = y^3 - 2x^3y + x^7.
$$

In [ ]:
# import the main Abelfunctions functionality
from abelfunctions import (
    RiemannSurface,
    AbelMap,
    RiemannConstantVector,
    Jacobian,
    puiseux,
    RiemannTheta,
)

# construct a Sage polynomial ring and the above curve
R.<x,y> = QQ[]
f = y**3 - 2*x**3*y + x**7
f = x**2*y**3 - x**4 + 1  # a genus 4 example
#f = x**2*y**3 - x**4 + y
#ftrott = 144*(x**4 + y**4) - 225*(x**2 + y**2) + 350*x**2*y**2 + 81
#f = ftrott
#fdividing = -180*x**5 + 396*y*x**4 - 307*x**3*y**2 + 107*x**2*y**3 + 273*x**3 - 318*x**2*y - 17*x*y**4 + 117*x*y**2 - 68*x + y**5 - 12*y**3 + 19*y   
#f = fdividing

# construct the corresponding Riemann surface
X = RiemannSurface(f)
print X
print X.genus()

In [ ]:
X.branch_points

# Demo - Places and Divisors

Computing the places "above" $x=0$ on the underlying curve, $C: f(x,y) = 0$.

$$P(0) = (0, \beta)$$

$x=0$ is a branch point of $f$ so one of these places is "ramified".

In [ ]:
b = X.branch_points; b

In [ ]:
places = X(0)
for P in places: print P

In [ ]:
P.puiseux_series.extend(16); P

In [ ]:
# roots of f(0,y)
#
print f(0,y)
print f(0,y).univariate_polynomial().roots()

In [ ]:
xt = P.puiseux_series.xpart
yt = P.puiseux_series.ypart

print xt
print yt
print
print f(xt,yt)

In [ ]:
P.puiseux_series.extend(20)

xt = P.puiseux_series.xpart
yt = P.puiseux_series.ypart

print xt
print yt
print
print f(xt,yt)

In [ ]:
X.branch_points

$x=1$ is not a branch point of $f$, so the places above $x=1$ are completely described by the roots of

$$f(1,y) = 0$$

In [ ]:
places = X(1)

for P in places:
    print P

We can still request the puiseux series representations at these places.

In [ ]:
puiseux(f,1)

In [ ]:
for P in puiseux(f,1):
    P.extend(4)
    print P
    print

Computing the place at $x=\infty$. (In this example there is exactly one.)

In [ ]:
P_oo = X('oo')[0]
P_oo.puiseux_series.extend(10)

print P_oo

In [ ]:
P_oo.puiseux_series.extend(32)

xt = P_oo.puiseux_series.xpart
yt = P_oo.puiseux_series.ypart

print f(xt,yt)

In [ ]:
print f(xt,yt).valuation()  ## f(xt,yt) = a*t^{valuation} + higher order terms

# Demo - Homology

Computing a canonical basis of cycles on the Riemann surface, $X$.

We can plot their projections into the affine plane.

In [ ]:
a = X.a_cycles()
b = X.b_cycles()

print a[0]

In [ ]:
# plot the first a-cycle using 512 equally-spaced points
#
# complex x-projection:
a[0].plot_x(512)

In [ ]:
# complex y-projection
#
a[0].plot_y(512, color='green')

In [ ]:
print 'x-values:'
print a[0].get_x(0)
print a[0].get_x(1.0)

print 'y-values:'
print a[0].get_y(0)[0]
print a[0].get_y(1.0)[0]

# Demo - Holomorphic Differentials

In [ ]:
omega = X.holomorphic_differentials()
for om in omega:
    print om

Compute the places above $x=0$ on $X$ and "localize" the differentials at that place.

One place $P \in X$ above $x=0$ looks like

$$P = \left(t, \frac{t^4}{2} + \frac{t^9}{16} + O(t^{11})\right)$$

In [ ]:
# localizing at a place
#
places = X(0)
P = places[0]
P.puiseux_series.extend(8)
print P

In [ ]:
omega[0].localize(P)

In [ ]:
omega[1].localize(P)

Another place $Q \in X$ above $x=0$ looks like

$$Q = \left(\frac{t^2}{2}, \frac{t^3}{2} + \frac{t^5}{32} + O(t^{12})\right)$$

In [ ]:
Q = places[1]
Q.puiseux_series.extend(12)
print Q

In [ ]:
omega[0].localize(Q)

In [ ]:
omega[1].localize(Q)

Computing the valuation divisors of these differentials:

In [ ]:
D1 = omega[0].valuation_divisor()
print D1

In [ ]:
D2 = omega[1].valuation_divisor()
print D2

**Fact:** If $\mathcal{C}$ is canonical then $\text{deg} \mathcal{C} = 2g - 2$

In [ ]:
g = X.genus()
D1.degree == 2*g-2

In [ ]:
D2.degree == 2*g-2

We can plot these differentials along a path $\gamma \subset X$.

For example, the $a$- and $b$-cycles:

In [ ]:
#omega[0].plot(a[0])

# Demo - Period Matrices

Integrating each of the holomorphic differentials around each of the a- and b-cycles.

In [ ]:
g = X.genus()
tau = X.period_matrix()

A = tau[:,:g]
B = tau[:,g:]

print A
print
print B

In [ ]:
a[0].integrate(omega[0])

In [ ]:
Omega = X.riemann_matrix()

print Omega

In [ ]:
from numpy.linalg import norm, eigvals

print 'symmetric?    ', norm(Omega - Omega.T)
print 'pos. definite?', eigvals(Omega.imag)

# Demo - The Abel Map

Given a fixed *base place* $P_0 \in X$, the Abel Map $A : X \to J(X)$ is defined

$$A(P) = \left( \int_{P_0}^P \omega_1, \ldots, \int_{P_0}^P \omega_g \right)$$

If $\mathcal{D} = \sum_i n_iP_i$ is a *divisor* on $X$ then

$$
A(\mathcal{D}) = \sum_i n_i A(P_i).
$$

In [ ]:
# pick two places
P = X(0)[0]
Q = X(I)[0]

# construct a divisor
D = 3*P + Q

In [ ]:
X.base_place

In [ ]:
J = Jacobian(X)   # reduces vectors modulo lattice ZZ^g + Omega ZZ^g
z1 = AbelMap(P)   # Abel map from P0 to P
z2 = AbelMap(Q)   # Abel map from P0 to Q
z3 = AbelMap(P,Q) # Abel map from P to Q
print z1
print z2
print z3

# numerically verify that A(P,Q) = A(P0,Q) - A(P0,P)
import numpy
print
print numpy.linalg.norm( J((z2-z1) - z3) )

In [ ]:
AbelMap(D)

In [ ]:
Q

In [ ]:
gamma = X.path(Q)
gamma.plot_x()

In [ ]:
gamma.plot_y(1000, color='green')

## Riemann Constant Vector

The Riemann constant vector satisfies the following two theorems:

**Theorem 1:** $\mathcal{C}$ is a canonical divisor if any only if $K(P_0) \equiv -A(P_0,\mathcal{C})$.

**Theorem 2:** $\theta(W,\Omega) = 0$ if and only if $W = A(P_0,\mathcal{D}) + K(P_0)$ where $\mathcal{D}$ is an effective degree $g-1$ divisor.

We compute $K$ below and verify that these two theorems are satisfied.

In [ ]:
K = RiemannConstantVector # alias the RCV function for brevity
P0 = X.base_place
print K(P0)

In [ ]:
C = omega[0].valuation_divisor()

In [ ]:
J = Jacobian(X)
z = J(2*K(P0) + AbelMap(C))
print z

In [ ]:
W = K(P0)
v = RiemannTheta.oscillatory_part(W,Omega)
print abs(v)

In [ ]:
D = X(2)[0]
W = J(AbelMap(D) + K(P0))
v = RiemannTheta.oscillatory_part(W, Omega)
print abs(v)